# **🧠 Logical Modeling: Solving Sudoku using Propositional and First-Order Logic with GUI**

This Python application solves Sudoku puzzles using **two logical paradigms**:

* **Propositional Logic (PL)** using a SAT solver
* **First-Order Logic (FOL)** using a recursive backtracking algorithm

It also provides a **graphical interface built with Tkinter** for user interaction.

---

## 🧩 **What It Does**

* Loads a **partially filled 9×9 Sudoku grid**.
* Presents a user-friendly **GUI** where the initial puzzle is displayed.
* Offers two solving methods:

  * **SAT-based (Propositional Logic)**: Encodes Sudoku as a Boolean satisfiability problem and solves it using the `Glucose3` SAT solver from the `pysat` library.
  * **FOL-based (Backtracking)**: Implements a traditional recursive depth-first search to fill in the valid digits.

---

## 🔑 **Core Concepts & Components**

### 1. **Variable Mapping for CNF Encoding**

```python
def var(r, c, n):
    return 81 * (r - 1) + 9 * (c - 1) + (n - 1) + 1
```

* Encodes a triple (row, col, number) into a unique variable for SAT solving.
* Ensures each logical clause in CNF can reference a distinct boolean variable.

---

### 2. **Sudoku Constraints as CNF**

The `sudoku_cnf()` function converts Sudoku rules into CNF clauses:

* **Cell constraint**: Each cell must contain exactly one number.
* **Row constraint**: Each number appears only once per row.
* **Column constraint**: Each number appears only once per column.
* **Block constraint**: Each number appears only once per 3×3 sub-grid.
* **Initial clues**: Already filled cells are enforced as unit clauses.

---

### 3. **SAT-Based Solver (PL)**

```python
def solve_with_pl(self):
    ...
    solver = Glucose3()
    solver.append_formula(cnf)
    ...
```

* Transforms the grid into CNF form.
* Uses Glucose3 to find a satisfying assignment.
* Extracts and maps variables back to grid values to reconstruct the solution.

---

### 4. **Backtracking Solver (FOL)**

```python
def solve_fol(grid):
    ...
```

* Uses First-Order Logic principles (valid number placement based on constraints).
* Implements a recursive trial-and-error method.
* Simpler but less efficient than SAT for large or complex grids.

---

## 🖼️ **Graphical Interface (Tkinter)**

### `SudokuGUI` Class:

* Dynamically creates 9×9 entry fields in a grid layout.
* Preloads a sample Sudoku puzzle (`initial_grid`) with some cells filled and locked (read-only).
* Provides two buttons to trigger either the PL or FOL solver.

### GUI Features:

* Entries with pre-filled values are black and uneditable.
* Computed values (solution) appear in **red**.
* Errors (unsolvable grids) trigger popup messages via `messagebox`.

---

## 🧰 **Libraries Used**

| Library         | Purpose                                  |
| --------------- | ---------------------------------------- |
| `tkinter`       | GUI creation                             |
| `pysat.formula` | Representing logical formulas in CNF     |
| `pysat.solvers` | Solving CNF formulas with Glucose3       |
| `copy`          | Deep copying grid for backtracking logic |

---

## 🧠 **Why Two Approaches?**

| Aspect                | Propositional Logic (PL)           | First-Order Logic (FOL)              |
| --------------------- | ---------------------------------- | ------------------------------------ |
| **Speed**             | Fast with powerful SAT solver      | Slower due to recursive backtracking |
| **Scalability**       | Good for harder puzzles            | May become slow for complex inputs   |
| **Complexity**        | Requires CNF encoding              | Conceptually simpler                 |
| **Educational Value** | Shows how logic can model problems | Demonstrates brute-force reasoning   |

---

## ✅ **Conclusion**

This project showcases a powerful combination of **logic-based problem modeling** and **user interaction**. By bridging **theoretical logic techniques** with practical application, it becomes a great learning tool for understanding:

* SAT problem encoding
* Recursive algorithms
* GUI development
* The distinction between Propositional and First-Order Logic in problem solving

---




In [1]:
import tkinter as tk
from tkinter import messagebox
from pysat.formula import CNF
from pysat.solvers import Glucose3
import copy

def var(r, c, n):
    return 81 * (r - 1) + 9 * (c - 1) + (n - 1) + 1

def sudoku_cnf(grid):
    cnf = CNF()

    for r in range(1, 10):
        for c in range(1, 10):
            cnf.append([var(r, c, n) for n in range(1, 10)])
            for n1 in range(1, 10):
                for n2 in range(n1 + 1, 10):
                    cnf.append([-var(r, c, n1), -var(r, c, n2)])

    for r in range(1, 10):
        for n in range(1, 10):
            cells = [var(r, c, n) for c in range(1, 10)]
            cnf.append(cells)
            for c1 in range(1, 10):
                for c2 in range(c1 + 1, 10):
                    cnf.append([-var(r, c1, n), -var(r, c2, n)])

    for c in range(1, 10):
        for n in range(1, 10):
            cells = [var(r, c, n) for r in range(1, 10)]
            cnf.append(cells)
            for r1 in range(1, 10):
                for r2 in range(r1 + 1, 10):
                    cnf.append([-var(r1, c, n), -var(r2, c, n)])

    for br in range(0, 3):
        for bc in range(0, 3):
            for n in range(1, 10):
                cells = []
                for r in range(1 + 3 * br, 4 + 3 * br):
                    for c in range(1 + 3 * bc, 4 + 3 * bc):
                        cells.append(var(r, c, n))
                cnf.append(cells)
                for i in range(len(cells)):
                    for j in range(i + 1, len(cells)):
                        cnf.append([-cells[i], -cells[j]])

    for r in range(9):
        for c in range(9):
            n = grid[r][c]
            if n != 0:
                cnf.append([var(r + 1, c + 1, n)])

    return cnf

def is_valid(board, row, col, num):
    if num in board[row]:
        return False
    for i in range(9):
        if board[i][col] == num:
            return False
    start_row = (row // 3) * 3
    start_col = (col // 3) * 3
    for i in range(start_row, start_row + 3):
        for j in range(start_col, start_col + 3):
            if board[i][j] == num:
                return False
    return True

def solve_fol(grid):
    for row in range(9):
        for col in range(9):
            if grid[row][col] == 0:
                for num in range(1, 10):
                    if is_valid(grid, row, col, num):
                        grid[row][col] = num
                        if solve_fol(grid):
                            return True
                        grid[row][col] = 0
                return False
    return True

class SudokuGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Sudoku Solver - PL and FOL")
        self.entries = [[None for _ in range(9)] for _ in range(9)]

        self.initial_grid = [
            [0, 1, 0, 0, 4, 0, 0, 5, 0],
            [4, 0, 7, 0, 0, 0, 6, 0, 2],
            [8, 2, 0, 6, 0, 0, 0, 7, 4],
            [0, 0, 0, 0, 1, 0, 5, 0, 0],
            [5, 0, 0, 0, 0, 0, 0, 0, 3],
            [0, 0, 4, 0, 5, 0, 0, 0, 0],
            [9, 6, 0, 0, 0, 3, 0, 4, 5],
            [3, 0, 5, 0, 0, 0, 8, 0, 1],
            [0, 7, 0, 0, 2, 0, 0, 3, 0]
        ]

        for i in range(9):
            for j in range(9):
                e = tk.Entry(root, width=2, font=('Arial', 18), justify='center')
                e.grid(row=i, column=j, padx=2, pady=2)
                val = self.initial_grid[i][j]
                if val != 0:
                    e.insert(0, str(val))
                    e.config(fg='black', state='disabled')
                self.entries[i][j] = e

        tk.Button(root, text="Solve with Propositional Logic", command=self.solve_with_pl).grid(row=9, column=0, columnspan=4, sticky="we", pady=5)
        tk.Button(root, text="Solve with First-Order Logic", command=self.solve_with_fol).grid(row=9, column=5, columnspan=4, sticky="we", pady=5)

    def get_grid(self):
        grid = []
        for i in range(9):
            row = []
            for j in range(9):
                val = self.entries[i][j].get()
                row.append(int(val) if val.isdigit() else 0)
            grid.append(row)
        return grid

    def display_solution(self, grid):
        for i in range(9):
            for j in range(9):
                entry = self.entries[i][j]
                if entry['state'] != 'disabled':
                    entry.delete(0, tk.END)
                    entry.insert(0, str(grid[i][j]))
                    entry.config(fg='red')

    def solve_with_pl(self):
        grid = self.get_grid()
        cnf = sudoku_cnf(grid)
        solver = Glucose3()
        solver.append_formula(cnf)

        if solver.solve():
            model = solver.get_model()
            result = [[0] * 9 for _ in range(9)]
            for r in range(1, 10):
                for c in range(1, 10):
                    if grid[r-1][c-1] != 0:
                        result[r-1][c-1] = grid[r-1][c-1]
                    else:
                        for n in range(1, 10):
                            if var(r, c, n) in model:
                                result[r-1][c-1] = n
                                break
            self.display_solution(result)
        else:
            messagebox.showerror("Error", "No solution found using Propositional Logic.")

    def solve_with_fol(self):
        grid = copy.deepcopy(self.get_grid())
        if solve_fol(grid):
            self.display_solution(grid)
        else:
            messagebox.showerror("Error", "No solution found using First-Order Logic.")

if __name__ == "__main__":
    root = tk.Tk()
    gui = SudokuGUI(root)
    root.mainloop()


## 🧪 **Extending the Project**

You could enhance this project with:

* Input feature for custom puzzles.
* Visualization of the solving steps.
* Sudoku generator with varying difficulty levels.
* Performance comparison (timing both solvers).
* Integration of more SAT solvers or FOL theorem provers.
